In [3]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Load Dataset
df = pd.read_csv(r"C:\Users\Shaik Shafiya\Downloads\sample_sales_data.csv")


# Basic Data Preprocessing
df['Sales'] = df['Sales'].astype(float)
df['Revenue'] = df['Sales'] * df['ProfitMargin']
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.to_period('M').astype(str)

# Initialize Dash App
app = dash.Dash(__name__)
app.title = "Sales Analytics Dashboard"

# Layout
app.layout = html.Div([
    html.H1(" Sales Data Analytics Dashboard", style={'textAlign': 'center'}),

    dcc.Dropdown(
        id='region-dropdown',
        options=[{'label': region, 'value': region} for region in df['Region'].unique()],
        value=df['Region'].unique()[0],
        clearable=False
    ),

    dcc.Graph(id='sales-over-time'),

    dcc.Graph(id='top-products'),

    dcc.Graph(id='revenue-by-category'),

    html.Div(id='summary-stats', style={'marginTop': 20})
])

# Callbacks
@app.callback(
    [Output('sales-over-time', 'figure'),
     Output('top-products', 'figure'),
     Output('revenue-by-category', 'figure'),
     Output('summary-stats', 'children')],
    [Input('region-dropdown', 'value')]
)
def update_dashboard(selected_region):
    filtered_df = df[df['Region'] == selected_region]

    # Sales over time
    fig1 = px.line(filtered_df.groupby('Month')['Sales'].sum().reset_index(),
                   x='Month', y='Sales', title='Monthly Sales Trend')

    # Top products
    top_products = filtered_df.groupby('Product')['Sales'].sum().nlargest(10).reset_index()
    fig2 = px.bar(top_products, x='Product', y='Sales', title='Top 10 Products')

    # Revenue by Category
    rev_cat = filtered_df.groupby('Category')['Revenue'].sum().reset_index()
    fig3 = px.pie(rev_cat, names='Category', values='Revenue', title='Revenue Share by Category')

    # Summary
    total_sales = filtered_df['Sales'].sum()
    avg_margin = filtered_df['ProfitMargin'].mean()
    summary = html.Div([
        html.H4(f"Total Sales in {selected_region}: ${total_sales:,.2f}"),
        html.H4(f"Average Profit Margin: {avg_margin:.2%}")
    ])

    return fig1, fig2, fig3, summary

# Run App
if __name__ == '__main__':
    app.run(debug=True)
